In [ ]:
!pip install icevision[all] > /dev/null

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from PIL import Image
import glob
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from icevision.all import *

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
PATH = "../input/militaryaircraftdetectiondataset/dataset/dataset/"
images = glob.glob(PATH + '*.jpg')
annot = []
for img in images:
    annot.append(img.replace('.jpg','.csv'))

df = pd.DataFrame({'image':images,'annot':annot})
df.head()

In [ ]:
def show_sample_image(n=5):
    img_index = list(np.round((np.random.random(n))*len(df)))
    fig,ax = plt.subplots(1,n,figsize=(15,15))
    i = 0
    for idx in img_index:
        img = df.image.values[int(idx)]
        img = Image.open(img)
        ax[i].imshow(img)
        annot = pd.read_csv(df.annot.values[int(idx)])
        for a in range(len(annot)):
            xmin = annot.xmin.values[a]
            ymin = annot.ymin.values[a]
            height = annot.height.values[a]
            width = annot.width.values[a]
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')
            # Add the patch to the Axes
            ax[i].add_patch(rect)
        i = i+1
    plt.show()
    return None

In [ ]:
show_sample_image(n=3)

In [ ]:
dataset = pd.DataFrame(columns = ['img','xmin','ymin','label','width','height'])
dataset.head()

In [ ]:
for i in tqdm(range(len(df))):
    annot = pd.read_csv(df.annot.values[i])
    label = annot['class'].values
    xmin = annot.xmin.values
    ymin = annot.ymin.values
    width = annot.width.values
    height = annot.height.values
    img = str(df.image.values[i])
    inter_df = pd.DataFrame({'img':img,'label':label,'xmin':xmin,'ymin':ymin,'width':width,'height':height})
    dataset = dataset.append(inter_df)
        

In [ ]:
dataset.shape

In [ ]:
template_record = ObjectDetectionRecord()

In [ ]:
class MyParser(Parser):
    def __init__(self, template_record):
        super().__init__(template_record=template_record)

        self.df = dataset[:100]
        self.class_map = ClassMap(list(self.df['label'].unique()))

    def __iter__(self) -> Any:
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self, o) -> Hashable:
        return o.img

    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(o.img)
            record.set_img_size(ImgSize(width=o.width, height=o.height))
            record.detection.set_class_map(self.class_map)
            
        record.detection.add_bboxes([BBox.from_xyxy(o.xmin, o.ymin, o.xmin + o.width, o.ymin + o.height)])
        record.detection.add_labels([o.label])


In [ ]:
parser = MyParser(template_record)

In [ ]:
train_records, valid_records = parser.parse()

In [ ]:
parser.class_map

In [ ]:
show_record(train_records[0], display_label=False, figsize=(14, 10))

In [ ]:
image_size = 384
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=image_size, presize=512), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])

In [ ]:
# Datasets
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
# Show an element of the train_ds with augmentation transformations applied
samples = [train_ds[0] for _ in range(3)]
show_samples(samples, ncols=3)

In [ ]:
model_type = models.torchvision.retinanet 
backbone = model_type.backbones.resnet50_fpn

model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(parser.class_map))

In [ ]:
train_dl = model_type.train_dl(train_ds, batch_size=4, num_workers=4, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=4, num_workers=4, shuffle=False)

In [ ]:
model_type.show_batch(first(valid_dl), ncols=4)

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(20, 1e-4, freeze_epochs=1)

In [ ]:
model_type.show_results(model, valid_ds, detection_threshold=.15)